In [2]:
import pandas as pd
import pandas as pd
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Masking, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import model_from_json

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import sys

Using TensorFlow backend.


In [39]:
import csv 
import requests 
import xml.etree.ElementTree as ET 
import pandas as pd
def parsegene(xmlfile): 
    # create element tree object 
    tree = ET.parse(xmlfile) 
    # get root element 
    root = tree.getroot()
    # create empty list for news items 
    geneitems = [] 
    # iterate news items 
    for item in root.findall('./topic/gene'): 
        gene = item.text
        # append news dictionary to news items list 
        geneitems.append(gene)  
    # return news items list 
    for i in range(19,21):
        geneitems.remove(geneitems[i])
    genes = str(geneitems)
    genes = re.sub("[a-x]|\[|\]|\'|\,|\(|\)|\W\d+\W|[y].","", genes).split(" ")
    geneslist = []                
    for i in genes:
        if i != "":
            geneslist.append(i)
    pattern = "|".join(geneslist)
    return pattern


nltk.download('stopwords')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
def clean_text(text):
    genes = parsegene("Topics/topics2018.xml")
    ## Remove puncuation
    text = str(text)
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    # Clean the text
    text = re.sub(r"\[\D\S{6}:\S{5}\]", "", text)
    text = re.sub(r"\/", "", text)
    text = re.sub(r"\\n*", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r'\,|\.|\:|\&', "", text)
    text = re.sub(r'\{|\}|\[|\]|\(|\)', "", text)
    text = re.sub(genes, "gene", text)
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = [wn.lemmatize(word) for word in text]
    text = " ".join(stemmed_words)
    return text


vocabulary_size = 20000
def process(df):
    tokenizer = Tokenizer(num_words= vocabulary_size)
    tokenizer.fit_on_texts(df['Abstracts'])
    sequences = tokenizer.texts_to_sequences(df['Abstracts'])
    data = pad_sequences(sequences, maxlen=500)
    df_save = pd.DataFrame(data)
    return df_save

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vly2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vly2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
test_x.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0,0,0,0,0,0,0,0,0,0,...,13,48,10,194,127,88,170,1544,1821,28
1,0,0,0,0,0,0,0,0,0,0,...,4,60,53,389,222,972,404,53,389,415
2,0,0,0,0,0,0,0,0,0,0,...,2,59,1132,1162,483,112,162,133,263,2281
3,0,0,0,0,0,0,0,0,0,0,...,3,10,385,183,107,644,477,85,1,2
4,0,0,0,0,0,0,0,0,0,0,...,107,3,13,48,58,536,220,134,88,399


In [42]:
import sklearn
x_train, y_train = sklearn.datasets.load_svmlight_file("data/train.txt")
x_valid, y_valid = sklearn.datasets.load_svmlight_file("data/val.txt")
q_train = np.loadtxt('data/train.group')
q_valid = np.loadtxt('data/val.group')
import lightgbm as lgb
gbm = lgb.LGBMRanker()
bst = gbm.fit(x_train, y_train, group=q_train, eval_set=[(x_valid, y_valid)],
eval_group=[q_valid], eval_at=[1, 3], early_stopping_rounds=20, verbose=True,
callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1)])

[1]	valid_0's ndcg@1: 0.346667	valid_0's ndcg@3: 0.358287
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's ndcg@1: 0.586667	valid_0's ndcg@3: 0.54946
[3]	valid_0's ndcg@1: 0.666667	valid_0's ndcg@3: 0.664841
[4]	valid_0's ndcg@1: 0.713333	valid_0's ndcg@3: 0.698436
[5]	valid_0's ndcg@1: 0.873333	valid_0's ndcg@3: 0.751956
[6]	valid_0's ndcg@1: 0.853333	valid_0's ndcg@3: 0.763874
[7]	valid_0's ndcg@1: 0.86	valid_0's ndcg@3: 0.790466
[8]	valid_0's ndcg@1: 0.893333	valid_0's ndcg@3: 0.801564
[9]	valid_0's ndcg@1: 0.866667	valid_0's ndcg@3: 0.814153
[10]	valid_0's ndcg@1: 0.906667	valid_0's ndcg@3: 0.817691
[11]	valid_0's ndcg@1: 0.88	valid_0's ndcg@3: 0.814301
[12]	valid_0's ndcg@1: 0.9	valid_0's ndcg@3: 0.821229
[13]	valid_0's ndcg@1: 0.886667	valid_0's ndcg@3: 0.824357
[14]	valid_0's ndcg@1: 0.886667	valid_0's ndcg@3: 0.828715
[15]	valid_0's ndcg@1: 0.9	valid_0's ndcg@3: 0.832588
[16]	valid_0's ndcg@1: 0.9	valid_0's ndcg@3: 0.830205
[17]	valid_0's ndcg@1: 0.88

In [43]:
def predict(data):
    data = pd.read_json(path_or_buf = data).iloc[:,3:4].rename(columns={"_source":"Abstracts"})
    df = data.dropna()
    df['Abstracts'] = df['Abstracts'].map(lambda x: clean_text(x))
    test_x = process(df)
    preds = gbm.predict(test_x)
    return preds

In [44]:
data = 'data/supafile.txt'
prediction = predict(data)
prediction

array([-0.15110153, -0.06419284, -0.21651752, ..., -0.14455386,
       -0.05767295,  0.02914337])